In [1]:
import gymnasium as gym
#envs = gym.vector.make('LunarLander-v2', num_envs=NUM_ENVS)#, render_mode='human')
#envs = gym.vector.make('LunarLander-v2', num_envs=NUM_ENVS, render_mode='human')


In [2]:
import tensorflow as tf
import numpy as np
import random

# Define the CNN model
def create_model():
    inputs = tf.keras.Input(shape=(8,))
    x = tf.keras.layers.Dense(32, activation="relu")(inputs)
    x = tf.keras.layers.Dense(32, activation="relu")(x)
    outputs = tf.keras.layers.Dense(4, activation="relu")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="LunarLander")
    return model

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)



2023-06-12 22:16:49.118625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 22:16:50.270463: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
import random

class Environment:
    def __init__(self, NUM_ENVS, EPSILON = 0.2):
        self.num_envs = NUM_ENVS
        self.envs = envs = gym.vector.make('LunarLander-v2', num_envs=NUM_ENVS)#, render_mode='human')
        self.current_state, _ = self.envs.reset()
        self.epsilon = 0.2

    def sample(self, model):
        q_values = model(self.current_state) #get q values for current state
        action = np.argmax(q_values, axis=1) #get action that maximizes q-value #potential error?
        action = [random.choice(range(4)) if random.random() < self.epsilon else a for a in action] #choose epsilon greedy
        new_observation, reward, terminated, truncated, info = self.envs.step(action)
        old_observation = self.current_state

        self.current_state = new_observation
        return (old_observation, action, reward, new_observation)


In [4]:
class Buffer:
    def __init__(self, max_size):
        self.buffer = []
        self.max_size = max_size

    def add_to_buffer(self, samples):
        #unpack the different environments
        for j in range(len(samples[0])):
            self.buffer.append(np.array([samples[0][j], samples[1][j] , samples[2][j], samples[3][j]]))
        if len(self.buffer) > self.max_size:
            self.buffer = self.buffer[-self.max_size:]

    def sample_minibatch(self, batch_size):
        return np.array(random.sample(self.buffer, min(len(self.buffer), batch_size)))
        

In [5]:
TAU = 0.01
N = 5
K = 3
MINI_BATCH_SIZE = 4
CONVERGED = False
MAX_STEPS = 100000
GAMMA = 0.4


In [6]:
envs = Environment(5)

# Create the model
Q_theta = create_model()

# Copy the model
Q_target = tf.keras.models.clone_model(Q_theta)

kabuff = Buffer(1000)

STEPS = 0
while not CONVERGED and STEPS < MAX_STEPS:
    STEPS += 1
    print(STEPS)
    
    Q_target.set_weights((1 - TAU) * np.array(Q_target.get_weights()) + TAU * np.array(Q_theta.get_weights())) #polyak averaging
    for n in range(N):
        kabuff.add_to_buffer(envs.sample(Q_theta))
        for k in range(K):
            #sample s,a,r,s' minibatch from buffer
            minibatch = kabuff.sample_minibatch(MINI_BATCH_SIZE)

            new_states = minibatch[:,3]
            rewards = minibatch[:,2]
            actions = minibatch[:,1]
            old_states = minibatch[:,0]

            new_states = np.array([list(new_states)])[0]
            old_states = np.array([list(old_states)])[0]
            actions = np.array([list(actions)])[0]

            #print(new_states.shape)

            Q_target_values = Q_target(new_states)
            max_Q_target_values = np.array([max(action_values) for action_values in Q_target_values])
            target_q_values = rewards + GAMMA * max_Q_target_values

            with tf.GradientTape() as tape:
                predictions = Q_theta(old_states)
                #print(predictions)
                selected_q_values = tf.gather(predictions, actions, batch_dims=1)
                loss_value = tf.reduce_mean(tf.square(target_q_values - selected_q_values))
            gradients = tape.gradient(loss_value, Q_theta.trainable_variables)
            optimizer.apply_gradients(zip(gradients, Q_theta.trainable_variables))

1


/tmp/ipykernel_61493/3520805313.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Q_target.set_weights((1 - TAU) * np.array(Q_target.get_weights()) + TAU * np.array(Q_theta.get_weights())) #polyak averaging
/tmp/ipykernel_61493/2178899444.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.buffer.append(np.array([samples[0][j], samples[1][j] , samples[2][j], samples[3][j]]))


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


KeyboardInterrupt: 